In [1]:
import sys
sys.path.append('../../../')
from pprint import pprint
import time
import pickle
import importlib
import numpy as np

In [2]:
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter
from lib_grover import make_hit_list
from libs_qrem import DeltaFilter

/var/folders/vr/d82vcp297v78vk4sb0_nzzdc0000gn/T/ipykernel_40609/1419118169.py:1: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.mitigation.measurement import TensoredMeasFitter


In [3]:
nbit = 25 # change this value to get discretized result closer to analytical results # 3
max_iter = 10

In [4]:
with open("pkls/raw_hist_list_list.pkl", "rb") as f:
    raw_hist_list_list = pickle.load(f)
with open("pkls/meas_fitters.pkl", "rb") as f:
    meas_fitters = pickle.load(f)

In [5]:
delta_mitigator_list = []
for i in range(0, max_iter):
    delta_mitigator_list.append(DeltaFilter(nbit + 1, meas_fitters[i].cal_matrices))
    if i % 10 == 0:
        print("size", i, "finished")
print("length of delta_mitigator_list: ", len(delta_mitigator_list))

size 0 finished
length of delta_mitigator_list:  10


In [6]:
def get_info(mitigator):
    return {"exact_one_norm_of_reduced_inv_A": mitigator.exact_one_norm_of_reduced_inv_A(),
            "mitigated_hist": mitigator.mitigated_hist(),
            "times": mitigator.times(),
            "mitigation_stddev": mitigator.mitigation_stddev(norm_type = "exact"),
            }

In [7]:
delta_info_list_list = []
delta_hit_list_list = []
for i, raw_hist_list in enumerate(raw_hist_list_list):
    delta_info_list = []
    delta_hist_list = []
    for raw_hist in raw_hist_list:
        delta_hist_list.append( delta_mitigator_list[i].apply(raw_hist) )
        delta_info_list.append( get_info(delta_mitigator_list[i]) )
    delta_hit_list_list.append( make_hit_list(delta_hist_list) )
    delta_info_list_list.append( delta_info_list )
    print("iter", i, "finished")

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished


In [8]:
with open("pkls/delta_info_list_list.pkl", "wb") as f:
    pickle.dump(delta_info_list_list, f)
with open("pkls/delta_hit_list_list.pkl", "wb") as f:
    pickle.dump(delta_hit_list_list, f)